# 1. Objetivo
O objetivo deste notebook é reproduzir a metodologia do cálculo de consumo aparente, apresentado no artigo [Indicadores de Consumo Aparente de Bens Industriais: metodologia e resultados](http://www.ipeadata.gov.br/doc/td_2101.pdf).

**Autor:** Amilton Lobo Mendes Júnior
amilton.mendes@gmail.com

# 2. Preparação do ambiente


## 2.1 Instalação de bibliotecas utilizadas

## 2.2. Import das bibliotecas utilizadas neste notebook





In [1]:
import pandas as pd
import requests
import json
from pandas import json_normalize
from datetime import datetime


## 2.3 URL das tabelas de recursos e usos de 2021, armazenada no github.

In [2]:
url_recursos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/dados/MIP/68_tab1_2019.xls'
url_usos = 'https://github.com/amiltonmendes/consumo-aparente/raw/refs/heads/main/dados/MIP/68_tab2_2019.xls'


In [ ]:

dados_movimento = pd.read_excel('dados/Funcex/Iqxbrcnae.xls',sheet_name='Mensal',skiprows=6,usecols=lambda col: col != 'Unnamed: 0' ,skipfooter=2)
dados_funcex_imp = pd.read_excel('dados/Funcex/Iqmbrcnae.xls',sheet_name='Mensal',skiprows=6,usecols=lambda col: col != 'Unnamed: 0' ,skipfooter=2)
# Transpor os dados a partir da terceira coluna
dados_movimento = dados_movimento.melt(
    id_vars=dados_movimento.columns[:1],  # Mantém as duas primeiras colunas como identificadores
    var_name="Mes",                   # Nome da nova coluna que conterá os títulos das colunas originais
    value_name="Valor"                # Nome da nova coluna que conterá os valores
)

dados_movimento = dados_movimento.rename(columns={'Unnamed: 1': 'Setor'})

dados_funcex_imp = dados_funcex_imp.melt(
    id_vars=dados_funcex_imp.columns[:1],  # Mantém as duas primeiras colunas como identificadores
    var_name="Mes",                   # Nome da nova coluna que conterá os títulos das colunas originais
    value_name="Valor"                # Nome da nova coluna que conterá os valores
)
dados_funcex_imp = dados_funcex_imp.rename(columns={'Unnamed: 1': 'Setor'})
dados_movimento['Mes'] = pd.to_datetime(dados_movimento['Mes'])
dados_movimento['Ano'] = dados_movimento['Mes'].dt.year
dados_movimento['Mes'] = dados_movimento['Mes'].dt.month

# Exibir o DataFrame transformado
dados_movimento.head()

,Setor,Mes,Valor
0,Agricultura e pecuária,2020-01-01 00:00:00,54.93
1,Produção florestal,2020-01-01 00:00:00,174.77
2,Pesca e aqüicultura,2020-01-01 00:00:00,89.30
3,Extração de petróleo e gás natural,2020-01-01 00:00:00,87.99
4,Extração de minerais metálicos,2020-01-01 00:00:00,79.08


In [ ]:
dados_movimento['Mes'] = pd.to_datetime(dados_movimento['Mes'])
dados_movimento['Ano'] = dados_movimento['Mes'].dt.year
dados_movimento['Mes'] = dados_movimento['Mes'].dt.month

0       1
1       1
2       1
3       1
4       1
       ..
1793    2
1794    2
1795    2
1796    2
1797    2
Name: Mes, Length: 1798, dtype: int32

In [48]:
url_pimpf = 'https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/202412,202501,202502/c544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342/d/v12606%205'

In [ ]:
r = requests.get(url_pimpf)
#Apresentar data de carregamento
a = json.loads(r.text)
res = json_normalize(a)

producao = pd.DataFrame(res)
nomes_colunas = producao.iloc[0].values
producao.columns = nomes_colunas
producao = producao.iloc[1:]
producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
producao['Ano'] = producao['Mês (Código)'].str[:4]
producao['Mes'] = producao['Mês (Código)'].str[4:]

producao['Ano'] = producao['Ano'].astype(int)
producao['Mes'] = producao['Mes'].astype(int)

#producao = producao[['Ano','Mes','Valor']]
producao = producao.rename(columns={'Valor' : 'Producao', 'Seções e atividades industriais (CNAE 2.0)' : 'Setor'})

producao['Producao'] = producao['Producao'].astype(float)
producao = producao [['Setor','Ano','Mes','Producao']]
producao['CNAE2D'] = producao['Setor'].str[2:4]


In [ ]:
#producao['CNAE2D']
producao['CNAE2D'] = producao['Setor'].str[2:4]

1     20
2     10
3     11
4     12
5     13
      ..
68    29
69    30
70    31
71    32
72    33
Name: Setor, Length: 72, dtype: object

In [54]:
producao

,Setor,Ano,Mes,Producao
1,3.20 Fabricação de produtos químicos,2024,12,90.35121
2,3.10 Fabricação de produtos alimentícios,2024,12,93.33385
3,3.11 Fabricação de bebidas,2024,12,111.59063
4,3.12 Fabricação de produtos do fumo,2024,12,47.50298
5,3.13 Fabricação de produtos têxteis,2024,12,85.83669
...,...,...,...,...
68,"3.29 Fabricação de veículos automotores, reboq...",2025,2,106.12670
69,3.30 Fabricação de outros equipamentos de tran...,2025,2,122.27612
70,3.31 Fabricação de móveis,2025,2,101.70610
71,3.32 Fabricação de produtos diversos,2025,2,87.58732


In [52]:
producao['Setor'].unique()

array(['3.20 Fabricação de produtos químicos',
       '3.10 Fabricação de produtos alimentícios',
       '3.11 Fabricação de bebidas',
       '3.12 Fabricação de produtos do fumo',
       '3.13 Fabricação de produtos têxteis',
       '3.14 Confecção de artigos do vestuário e acessórios',
       '3.15 Preparação de couros e fabricação de artefatos de couro, artigos para viagem e calçados',
       '3.16 Fabricação de produtos de madeira',
       '3.17 Fabricação de celulose, papel e produtos de papel',
       '3.18 Impressão e reprodução de gravações',
       '3.19 Fabricação de coque, de produtos derivados do petróleo e de biocombustíveis',
       '3.21 Fabricação de produtos farmoquímicos e farmacêuticos',
       '3.22 Fabricação de produtos de borracha e de material plástico',
       '3.23 Fabricação de produtos de minerais não metálicos',
       '3.24 Metalurgia',
       '3.25 Fabricação de produtos de metal, exceto máquinas e equipamentos',
       '3.26 Fabricação de equipamentos de

In [8]:
correlacao = pd.read_excel('dados/Correlação SCN PIMPF FUNCEX.xlsx')

In [9]:
correlacao.head()

,Prefixo Cód. SCN,PIM-PF,Funcex
0,10,3.10 Fabricação de produtos alimentícios,Produtos alimentícios
1,11,3.11 Fabricação de bebidas,Bebidas
2,12,3.12 Fabricação de produtos do fumo,Produtos do fumo
3,13,3.13 Fabricação de produtos têxteis,Produtos têxteis
4,14,3.14 Confecção de artigos do vestuário e acess...,Confecção de artigos do vestuário e acessórios


## 2.4 Fórmula

1. Inicialmente, foram carregados os dados de produção e importação da tabela de recursos (Tabela 1 das TRU's).
2. Em seguida, tanto para importação quanto para produção foram somados todos os z produtos pertencentes a um mesmo setor, conforme fórmula abaixo:

$$
    \sum_{n=1}^{z}a_n
$$
3. Com isso, obtemos os valores totais de produção e importação por setor a preços básicos, restando as informações de exportação, que são carregados da tabela de usos (Tabela 2 das TRU's).

4. A exportação está disponibilizada a preços do consumidor, ou seja, com o acréscimo de margem de transporte, margem de comércio e impostos, que devem ser subtraídos a fim de que tenhamos uma informação comparável com os valores de importação e produção, ou seja, a preços básicos. Para isso, se faz necessário calcular o percentual que o preço básico representa do preço do consumidor. Essa razão é o fator de ajuste, que é multiplicado ao valor de exportação.

5. De posse dos valores de exportação, importação e produção das contas nacionais, obtêm-se a participação de cada uma dessas variáveis de acordo com as fórmulas:

$$

\\
w_p = \frac{P}{P-X+I}
\\
w_x = \frac{X}{P-X+I}
\\
w_i = \frac{I}{P-X+I}

$$



**Onde:**
$$
I : \text{Valor de importação do setor}
\\
P : \text{Valor de produção do setor}
\\
X : \text{Valor de exportação do setor}
$$

6. Por fim os fatores de ponderação, são aplicados a dados de exportação e importação ( quantum calculados pela funcex) e produção ( pim-pf ) após eles serem convertidos a uma mesma base ( ano de referência). Com isso, calcula-se o consumo aparente.

## 2.5 Funções e classes utilizadas no notebook

In [ ]:
#Carrega dados oferta
def carrega_dados_oferta():
  oferta = pd.read_excel(url_recursos, sheet_name='oferta',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  #Renomear as duas primeiras colunas que não foram carregadas por estarem mescladas
  oferta= oferta.rename(columns={oferta.columns[0] : 'Código do produto', oferta.columns[1] : 'Descrição do produto'})
  #Remover a primeira linha (vazia)
  oferta = oferta.iloc[1:]
  return oferta
def carrega_dados_producao():
  producao = pd.read_excel(url_recursos, sheet_name='producao',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  producao = producao.rename(columns={producao.columns[0] : 'Código do produto', producao.columns[1] : 'Descrição do produto'})
  producao = producao.iloc[1:]
  return producao

def carrega_dados_importacao():
  importacao = pd.read_excel(url_recursos, sheet_name='importacao',skiprows=3,skipfooter=5,dtype={'Unnamed: 0' : str})
  importacao = importacao.rename(columns={importacao.columns[0] : 'Código do produto', importacao.columns[1] : 'Descrição do produto'})
  importacao = importacao.iloc[1:]
  return importacao

def carrega_dados_exportacao():
  exportacao = pd.read_excel(url_usos, sheet_name='demanda',skiprows=3,skipfooter=2,dtype={'Unnamed: 0' : str})
  exportacao = exportacao.rename(columns={exportacao.columns[0] : 'Código do produto', exportacao.columns[1] : 'Descrição do produto'})
  exportacao = exportacao.iloc[1:]
  return exportacao

class ConsumoAparente():
  def __init__(self,oferta,producao,importacao,exportacao):
    self.oferta = oferta
    self.producao = producao
    self.importacao = importacao
    self.exportacao = exportacao
  def carrega_dados_producao_mensal(self,url='https://apisidra.ibge.gov.br/values/t/8888/n1/all/v/12606/p/all/c544/56689,129317,129318,129319,129320,129321,129322,129323,129324,129325,129326,129330,129331,129332,129333,129334,129335,129336,129337,129338,129339,129340,129341,129342/d/v12606%203'):
    r = requests.get(url)
    #Apresentar data de carregamento
    a = json.loads(r.text)
    res = json_normalize(a)

    producao = pd.DataFrame(res)
    nomes_colunas = producao.iloc[0].values
    producao.columns = nomes_colunas
    producao = producao.iloc[1:]
    producao['Mês (Código)'] = producao['Mês (Código)'].astype(str)
    producao['Ano'] = producao['Mês (Código)'].str[:4]
    producao['Mes'] = producao['Mês (Código)'].str[4:]

    producao['Ano'] = producao['Ano'].astype(int)
    producao['Mes'] = producao['Mes'].astype(int)

    #producao = producao[['Ano','Mes','Valor']]
    producao = producao.rename(columns={'Valor' : 'Producao', 'Seções e atividades industriais (CNAE 2.0)' : 'Setor'})


    producao['Producao'] = pd.to_numeric(producao['Producao'],errors='coerce')
    
    producao = producao [['Setor','Ano','Mes','Producao']]
    producao['CNAE2D'] = producao['Setor'].str[2:4]

    return producao
  def carrega_dados_quantum_exp_imp(self,movimento):
    if movimento == 'exportacao':
      url = 'dados/Funcex/Iqxbrcnae.xls'
    elif movimento == 'importacao':
      url = 'dados/Funcex/Iqmbrcnae.xls'
    retorno = pd.DataFrame()
    for sheet in ['Mensal','Mensal 2010-2019']:
      dados_movimento = pd.read_excel(url,sheet_name=sheet,skiprows=6,usecols=lambda col: col != 'Unnamed: 0' ,skipfooter=2)
      # Transpor os dados a partir da terceira coluna
      dados_movimento = dados_movimento.melt(
          id_vars=dados_movimento.columns[:1],  # Mantém as duas primeiras colunas como identificadores
          var_name="Mes",                   # Nome da nova coluna que conterá os títulos das colunas originais
          value_name="Valor"                # Nome da nova coluna que conterá os valores
      )

      dados_movimento = dados_movimento.rename(columns={'Unnamed: 1': 'Setor'})
      dados_movimento['Mes'] = pd.to_datetime(dados_movimento['Mes'])
      dados_movimento['Ano'] = dados_movimento['Mes'].dt.year
      dados_movimento['Mes'] = dados_movimento['Mes'].dt.month

      dados_movimento = dados_movimento.rename(columns={'Valor' : movimento})
      dados_movimento[movimento] = dados_movimento[movimento].astype(float)

      retorno = pd.concat([retorno,dados_movimento],ignore_index=True)

    return retorno
  

  def recupera_valor_exportacao_scn(self,cnae):
    preco_consumidor,preco_basico = self.oferta[self.oferta['Código do produto'].str.startswith(cnae)][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].sum().values
    fator_ajuste = preco_basico/preco_consumidor
    valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].sum().values[0]
    valor_exportacao = valor_exportacao*fator_ajuste
    return valor_exportacao
  def recupera_valor_producao_scn(self,cnae):
    valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].sum().values[0]
    return valor_producao
  def recupera_valor_importacao_scn(self,cnae):
    valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)][['Importação de bens\ne serviços (1)']].sum().values[0]
    return valor_importacao
  
  def muda_base_indice(self,df,coluna,ano_base = 2012):
    #Converter esse índice para a base de selecionada
    medias_por_setor = df[df['Ano'] == ano_base].groupby('Setor')[coluna].mean().reset_index()
    medias_por_setor = medias_por_setor.rename(columns={coluna: f'{coluna}_media_base'})

    # Mesclar as médias calculadas com o DataFrame original
    df = df.merge(medias_por_setor, on='Setor', how='left')

    # Ajustar o índice para a nova base por setor
    df[f'{coluna}_b{ano_base}'] = df[coluna] * 100 / df[f'{coluna}_media_base']

    # Remover a coluna auxiliar de médias
    df = df.drop(columns=[f'{coluna}_media_base'])
    return df

  def calcula_consumo_aparente(self,lista_cnae,verbose=0,ano_base = 2012):
    #verificar se cnae é uma lista ou uma única string. Se for uma lista, faz um loop.
    valores = []
    correlacao = pd.read_excel('dados/Correlação SCN PIMPF FUNCEX.xlsx')

    indices_importacao = self.carrega_dados_quantum_exp_imp('importacao')
    indices_exportacao = self.carrega_dados_quantum_exp_imp('exportacao')
    indices_producao = self.carrega_dados_producao_mensal()

    
    #Dataframe com os índices mensais de producao, importação e exportação, com bases distintas e não ponderados, 
    # filtrados pelas cnaes passadas como parâmetro
    indices_consolidados = indices_producao.merge(correlacao[['PIM-PF','Funcex','Prefixo Cód. SCN']].rename(columns={'PIM-PF' : 'Setor'}),how='inner',on='Setor').merge(indices_importacao[['Setor','Ano','Mes','importacao']].rename(columns={'Setor' : 'Funcex'}),how='inner',on=['Ano','Mes','Funcex'])\
    .merge(indices_exportacao[['Setor','Ano','Mes','exportacao']].rename(columns={'Setor' : 'Funcex'}),how='inner',on=['Ano','Mes','Funcex'])
    
    indices_consolidados = self.muda_base_indice(indices_consolidados,'importacao',ano_base)
    indices_consolidados = self.muda_base_indice(indices_consolidados,'exportacao',ano_base)
    indices_consolidados = self.muda_base_indice(indices_consolidados,'Producao',ano_base)
    total_importacao = 0
    total_producao = 0
    total_exportacao = 0
    for cnae in indices_consolidados['Prefixo Cód. SCN'].unique():
      cnae = str(cnae)
      valor_importacao = self.recupera_valor_importacao_scn(cnae)
      valor_producao = self.recupera_valor_producao_scn(cnae)
      valor_exportacao = self.recupera_valor_exportacao_scn(cnae)
      valores.append({'Prefixo Cnae' : cnae, 'Importacao' : 'valor_importacao', 'Producao' : valor_producao, 'Exportacao' : valor_exportacao})
      total_importacao += valor_importacao
      total_producao += valor_producao
      total_exportacao += valor_exportacao
      
      if verbose == 1:
        print(f'Prefixo do código SCN: {cnae}')
        print(f'Valor da importação: {valor_importacao}')
        print(f'Valor da produção: {valor_producao}')
        print(f'Valor da exportação: {valor_exportacao}')
        
    return valores_importacao,valores_producao,valores_exportacao    


      

      

    valor_total_importacao = sum(valores_importacao)
    valor_total_producao = sum(valores_producao)
    valor_total_exportacao = sum(valores_exportacao)
    peso_importacao_setor = valor_importacao/valor_total_importacao
    peso_producao_setor = valor_producao/valor_total_producao
    peso_exportacao_setor = valor_exportacao/valor_total_exportacao

    total_setores = valor_total_producao + valor_total_importacao - valor_total_exportacao 
    peso_importacao = valor_total_importacao/total_setores
    peso_exportacao = valor_total_exportacao/total_setores
    peso_producao = valor_total_producao/total_setores




    #Prepara dados de quantum de exportação e importação para a junção com dados de produção e cálculo de consumo aparente
    if verbose == 1:
      print(f'Peso da importação: {peso_importacao}')
      print(f'Peso da exportação: {peso_exportacao}')
      print(f'Peso da produção: {peso_producao}')
      print('Carregando dados de produção da PIM-PF (Sidra)')
    producao = self.carrega_dados_producao_mensal()
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = producao[producao['Ano'] == ano_base]['producao'].mean()
    producao['producao'] = producao['producao']*100/media_indice_base_selecionada

    if verbose == 1:
      print('Carregando dados de exportação e importação da Quantum ( Funcex - Ipeadata)')
    dados_exportacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[lista_cnae]['exportacao'],'exportacao')
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = dados_exportacao[dados_exportacao['Ano'] == ano_base]['exportacao'].mean()
    dados_exportacao['exportacao'] = dados_exportacao['exportacao']*100/media_indice_base_selecionada
    dados_importacao = self.carrega_dados_quantum_exp_imp(mapa_cnae[lista_cnae]['importacao'],'importacao')
    #Converter esse índice para a base de selecionada
    media_indice_base_selecionada = dados_importacao[dados_importacao['Ano'] == ano_base]['importacao'].mean()
    dados_importacao['importacao'] = dados_importacao['importacao']*100/media_indice_base_selecionada

    #Juntar os dados de produção, importação e exportação
    producao = producao.merge(dados_exportacao,how='inner',on=['Ano','Mes']).merge(dados_importacao,how='inner',on=['Ano','Mes'])



    producao['producao'] = producao['producao'].astype(float)
    producao['importacao'] = producao['importacao'].astype(float)
    producao['exportacao'] = producao['exportacao'].astype(float)

    producao['consumo_aparente'] = producao['producao']*peso_producao - producao['exportacao']*peso_exportacao + producao['importacao']*peso_importacao

    producao['consumo_aparente_12_meses_anterior'] = producao['consumo_aparente'].shift(12)
    producao['variacao_consumo_aparente'] = ((producao['consumo_aparente']/producao['consumo_aparente_12_meses_anterior'])-1)*100

    producao['consumo_aparente_12m'] = producao['consumo_aparente'].rolling(window=12).sum()
    producao['producao_12m'] = producao['producao'].rolling(window=12).sum()
    producao['importacao_12m'] = producao['importacao'].rolling(window=12).sum()

    # Calcular a variação percentual em relação aos 12 meses anteriores
    producao['variacao_consumo_aparente_12m'] = (
        (producao['consumo_aparente_12m'] / producao['consumo_aparente_12m'].shift(12)) - 1
    ) * 100
    producao['variacao_producao_12m'] = (
        (producao['producao_12m'] / producao['producao_12m'].shift(12)) - 1
    ) * 100
    producao['variacao_importacao_12m'] = (
        (producao['importacao_12m'] / producao['importacao_12m'].shift(12)) - 1
    ) * 100

    # Arredondar os valores para 2 casas decimais
    producao['variacao_consumo_aparente_12m'] = producao['variacao_consumo_aparente_12m'].round(2)
    producao['variacao_producao_12m'] = producao['variacao_producao_12m'].round(2)
    producao['variacao_importacao_12m'] = producao['variacao_importacao_12m'].round(2)


    #Calcular a variação da produção e importação acumulada dos 12 meses anteriores



    return producao






In [175]:
consumoAparente = ConsumoAparente(carrega_dados_oferta(),carrega_dados_producao(),carrega_dados_importacao(),carrega_dados_exportacao())

In [176]:
consumoAparente.calcula_consumo_aparente(['21','29'],verbose=1,ano_base=2012)

Valor da importação: 156878.0
Valor da produção: 334552.0
Valor da exportação: 28221.26533499374
Valor da importação: 21551.0
Valor da produção: 719190.0
Valor da exportação: 99197.26506644464
Valor da importação: 10791.0
Valor da produção: 92496.0
Valor da exportação: 2472.7449134079243
Valor da importação: 9268.0
Valor da produção: 16645.0
Valor da exportação: 5047.333169475656
Valor da importação: 12534.0
Valor da produção: 59323.0
Valor da exportação: 1565.7650264536246
Valor da importação: 11215.0
Valor da produção: 69978.0
Valor da exportação: 1204.4451110559623
Valor da importação: 3826.0
Valor da produção: 42039.0
Valor da exportação: 4735.783587257618
Valor da importação: 486.0
Valor da produção: 33444.0
Valor da exportação: 8250.930818989145
Valor da importação: 4580.0
Valor da produção: 109387.0
Valor da exportação: 28809.301654594183
Valor da importação: 328.0
Valor da produção: 19153.0
Valor da exportação: 50.57715317755954
Valor da importação: 65593.0
Valor da produção: 5

([np.float64(156878.0),
  np.float64(21551.0),
  np.float64(10791.0),
  np.float64(9268.0),
  np.float64(12534.0),
  np.float64(11215.0),
  np.float64(3826.0),
  np.float64(486.0),
  np.float64(4580.0),
  np.float64(328.0),
  np.float64(65593.0),
  np.float64(38407.0),
  np.float64(21840.0),
  np.float64(7573.0),
  np.float64(44287.0),
  np.float64(16688.0),
  np.float64(91519.0),
  np.float64(33110.0),
  np.float64(71946.0),
  np.float64(65329.0),
  np.float64(50990.0),
  np.float64(2394.0),
  np.float64(15274.0)],
 [np.float64(334552.0),
  np.float64(719190.0),
  np.float64(92496.0),
  np.float64(16645.0),
  np.float64(59323.0),
  np.float64(69978.0),
  np.float64(42039.0),
  np.float64(33444.0),
  np.float64(109387.0),
  np.float64(19153.0),
  np.float64(562116.0),
  np.float64(80374.0),
  np.float64(120321.0),
  np.float64(87249.0),
  np.float64(212117.0),
  np.float64(114036.0),
  np.float64(103992.0),
  np.float64(90546.0),
  np.float64(145879.0),
  np.float64(305563.0),
  np.flo

In [172]:
preco_basico,preco_consumidor = consumoAparente.oferta[consumoAparente.oferta['Código do produto'].str.startswith('29')][['Oferta total\na preço\nde consumidor','Oferta total\n a preço básico ']].sum().values


In [173]:
preco_basico

np.float64(509561.0)

In [155]:
valor_importacao = consumoAparente.importacao[consumoAparente.importacao['Código do produto'].str.startswith('29')][['Importação de bens\ne serviços (1)']].sum().values[0]


In [156]:
valor_importacao

np.float64(65329.0)

In [139]:
teste['Funcex'].unique()

array(['Produtos químicos', 'Produtos alimentícios', 'Bebidas  ',
       'Produtos do fumo', 'Produtos têxteis',
       'Confecção de artigos do  vestuário e acessórios',
       'Couros, artefatos de couro, artigos para viagem e calçados',
       'Produtos de madeira', 'Celulose, papel e produtos de papel',
       'Impressão e reprodução de gravações',
       'Derivados do petróleo biocombustíveis e coque',
       'Produtos farmoquímicos farmacêuticos',
       'Produtos de borracha e de material plástico',
       'Produtos de minerais não-metálicos', 'Metalurgia  ',
       'Produtos de metal, exceto  máquinas e equipamentos',
       'Equipamentos de informática, produtos eletrônicos e ópticos',
       'Máquinas, aparelhos e materiais elétricos',
       'Máquinas e equipamentos',
       'Veículos automotores, reboques e carrocerias',
       'Outros equipamentos de transporte, exceto veículos automotores',
       'Móveis  ', 'Indústrias diversas'], dtype=object)

In [69]:
correlacao = pd.read_excel('dados/Correlação SCN PIMPF FUNCEX.xlsx')


In [70]:
correlacao.head()

,Prefixo Cód. SCN,PIM-PF,Funcex
0,10,3.10 Fabricação de produtos alimentícios,Produtos alimentícios
1,11,3.11 Fabricação de bebidas,Bebidas
2,12,3.12 Fabricação de produtos do fumo,Produtos do fumo
3,13,3.13 Fabricação de produtos têxteis,Produtos têxteis
4,14,3.14 Confecção de artigos do vestuário e acess...,Confecção de artigos do vestuário e acessórios


In [118]:
producao = consumoAparente.carrega_dados_producao_mensal()

In [119]:
producao['Ano'].min()

np.int64(2002)

In [71]:
producao = producao[producao['CNAE2D'].isin(['21','29'])]

In [74]:
importacao = consumoAparente.carrega_dados_quantum_exp_imp('importacao')

In [78]:
exportacao = consumoAparente.carrega_dados_quantum_exp_imp('exportacao')

In [79]:
exportacao

,Setor,Mes,exportacao,Ano
0,Agricultura e pecuária,1,54.93,2020
1,Produção florestal,1,174.77,2020
2,Pesca e aqüicultura,1,89.30,2020
3,Extração de petróleo e gás natural,1,87.99,2020
4,Extração de minerais metálicos,1,79.08,2020
...,...,...,...,...
1793,Máquinas e equipamentos,2,88.33,2025
1794,"Veículos automotores, reboques e carrocerias",2,88.81,2025
1795,"Outros equipamentos de transporte, exceto veíc...",2,47.08,2025
1796,Móveis,2,107.15,2025


In [72]:
producao

,Setor,Ano,Mes,Producao,CNAE2D
12,3.21 Fabricação de produtos farmoquímicos e fa...,2024,12,75.63582,21
20,"3.29 Fabricação de veículos automotores, reboq...",2024,12,88.82298,29
36,3.21 Fabricação de produtos farmoquímicos e fa...,2025,1,81.49361,21
44,"3.29 Fabricação de veículos automotores, reboq...",2025,1,90.06828,29
60,3.21 Fabricação de produtos farmoquímicos e fa...,2025,2,82.49477,21
68,"3.29 Fabricação de veículos automotores, reboq...",2025,2,106.12670,29


In [83]:
producao.merge(correlacao[['PIM-PF','Funcex','Prefixo Cód. SCN']].rename(columns={'PIM-PF' : 'Setor'}),how='inner',on='Setor').merge(importacao[['Setor','Ano','Mes','importacao']].rename(columns={'Setor' : 'Funcex'}),how='inner',on=['Ano','Mes','Funcex'])\
    .merge(exportacao[['Setor','Ano','Mes','exportacao']].rename(columns={'Setor' : 'Funcex'}),how='inner',on=['Ano','Mes','Funcex'])['Prefixo Cód. SCN'].unique()

array([21, 29])

In [43]:
#realizar a soma ponderada pela participação dos fatores de ajuste
peso_exportacao = valor_exportacao[0]*(valor_exportacao[0]/(sum(valor_exportacao))) + valor_exportacao[1]*(valor_exportacao[1]/(sum(valor_exportacao))) + valor_exportacao[2]*(valor_exportacao[2]/(sum(valor_exportacao)))
peso_exportacao



np.float64(11612.806094753409)

In [ ]:

fator_ajuste = preco_basico/preco_consumidor
valor_producao = self.producao[self.producao['Código do produto'].str.startswith(cnae)][['Total\ndo produto']].sum().values[0]
valor_exportacao = self.exportacao[self.exportacao['Código do produto'].str.startswith(cnae)][['Exportação\nde bens e\nserviços (1)']].sum().values[0]
valor_exportacao = valor_exportacao*fator_ajuste
valor_importacao = self.importacao[self.importacao['Código do produto'].str.startswith(cnae)][['Importação de bens\ne serviços (1)']].sum().values[0]
#Calcular pesos de acordo com fórmula constante do arquivo
total = valor_importacao - valor_exportacao + valor_producao
peso_importacao = valor_importacao/total
peso_exportacao = valor_exportacao/total
peso_producao = valor_producao/total


In [ ]:
consumoAparente.calcula_consumo_aparente('29',0,ano_base=2021).tail(13)[['consumo_aparente','variacao_consumo_aparente','variacao_consumo_aparente_12m','variacao_producao_12m','variacao_importacao_12m']]

,consumo_aparente,variacao_consumo_aparente,variacao_consumo_aparente_12m,variacao_producao_12m,variacao_importacao_12m
265,81.524671,10.868147,-3.40,-6.13,5.06
266,90.088503,-0.200384,-3.44,-6.66,5.48
267,101.636080,37.185664,-0.30,-3.90,7.68
268,93.385690,3.569473,-0.26,-4.70,9.41
269,108.411484,34.362234,2.79,-3.70,18.97
270,101.882934,31.939077,5.98,-0.83,21.00
271,108.057077,16.425363,8.25,1.33,22.84
272,102.096517,18.499054,11.19,4.47,24.65
273,114.153131,32.378826,14.36,7.66,26.93
274,98.637536,15.184462,16.80,10.55,27.46


In [ ]:
consumoAparente.calcula_consumo_aparente('2100',0,ano_base=2019).tail(13)[['consumo_aparente','variacao_consumo_aparente','consumo_aparente_12_meses_anterior']]

,consumo_aparente,variacao_consumo_aparente,consumo_aparente_12_meses_anterior
265,96.464852,-12.570714,110.334713
266,104.300955,-23.447959,136.248431
267,108.510679,-12.722958,124.329006
268,123.332733,6.922561,115.347717
269,114.492271,7.920777,106.089184
270,122.482596,14.753406,106.735477
271,121.850753,-0.630247,122.623585
272,110.187628,4.169732,105.777010
273,122.012365,3.551960,117.827190
274,107.944177,4.260677,103.532971


In [ ]:
producao = consumoAparente.carrega_dados_producao_mensal(mapa_cnae['2100']['sidra_url'])


In [ ]:
producao.tail(14)

,Ano,Mes,producao
265,2024,1,72.81993
266,2024,2,82.38201
267,2024,3,92.89157
268,2024,4,100.29527
269,2024,5,103.71688
270,2024,6,107.13063
271,2024,7,102.94700
272,2024,8,110.04800
273,2024,9,102.27364
274,2024,10,109.16742


In [ ]:
variacao_producao_12m = producao['producao'].pct_change(periods=12)*100
producao['variacao_producao_12m'] = variacao_producao_12m
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].round(2)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(float)
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].astype(str).str.replace('nan','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('inf','-')
producao['variacao_producao_12m'] = producao['variacao_producao_12m'].str.replace('-','-')



In [ ]:
producao.tail(14)

,Ano,Mes,producao,variacao_producao_12m
265,2024,1,72.81993,-17.24
266,2024,2,82.38201,-18.73
267,2024,3,92.89157,-16.41
268,2024,4,100.29527,-6.55
269,2024,5,103.71688,1.98
270,2024,6,107.13063,16.26
271,2024,7,102.94700,6.06
272,2024,8,110.04800,-6.72
273,2024,9,102.27364,3.37
274,2024,10,109.16742,4.68
